In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense , merge
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import ReduceLROnPlateau


from keras.layers.merge import dot
from keras.models import Model


# specifically for deeplearning.
from keras.layers import Dropout, Flatten,Activation,Input,Embedding
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf
import random as rn
from IPython.display import SVG
 
# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
#import cv2                  
import numpy as np  
#from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image


#TL pecific modules
from keras.applications.vgg16 import VGG16

In [2]:
train=pd.read_csv(r'./dataset/rating.csv')


In [3]:
rand_userIds = np.random.choice(train['userId'].unique(),size=int(len(train['userId'].unique())*0.1),replace=False)

train = train.loc[train['userId'].isin(rand_userIds)]

In [4]:
df=train.copy()

In [5]:
df.shape

(2008784, 4)

In [6]:
df.head()

,userId,movieId,rating,timestamp
517,6,1,5.0,1997-03-13 17:50:52
518,6,3,3.0,1997-03-13 17:52:38
519,6,7,5.0,1997-03-13 17:52:38
520,6,17,5.0,1997-03-13 17:50:52
521,6,52,5.0,1997-03-13 17:54:18


In [7]:
df['userId'].unique()

array([     6,     30,     62, ..., 138474, 138479, 138485])

In [8]:
len(df['userId'].unique())

13849

In [9]:
df['movieId'].unique()

array([     1,      3,      7, ..., 115941, 115945,  64754])

In [10]:
len(df['movieId'].unique())

17905

In [11]:
df['userId'].isnull().sum()

0

In [12]:
df['rating'].isnull().sum()

0

In [13]:
df['movieId'].isnull().sum()

0

In [14]:
df['rating'].min()

0.5

In [15]:
df['rating'].max()

5.0

In [16]:
df.userId = df.userId.astype('category').cat.codes.values


In [17]:
df.movieId = df.movieId.astype('category').cat.codes.values

In [18]:
df['userId'].value_counts(ascending=True)

4206       20
4256       20
13479      20
7330       20
6991       20
         ... 
5153     3541
13014    3908
6273     3958
1538     4354
13053    4785
Name: userId, Length: 13849, dtype: int64

In [19]:
df['movieId'].unique()

array([    0,     2,     6, ..., 17377, 17378, 11965], dtype=int16)

In [20]:
index=list(df['userId'].unique())
columns=list(df['movieId'].unique())
index=sorted(index)
columns=sorted(columns)

In [21]:
util_df=pd.pivot_table(data=df,values='rating',index='userId',columns='movieId')

In [22]:
util_df

movieId,0,1,2,3,4,5,6,7,8,9,...,17895,17896,17897,17898,17899,17900,17901,17902,17903,17904
userId,,,,,,,,,,,,,,,,,,,,,
0,5.0,NaN,3.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13845,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13846,5.0,4.0,NaN,NaN,4.0,5.0,NaN,NaN,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
util_df.fillna(0)

movieId,0,1,2,3,4,5,6,7,8,9,...,17895,17896,17897,17898,17899,17900,17901,17902,17903,17904
userId,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13845,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13846,5.0,4.0,0.0,0.0,4.0,5.0,0.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
util_df.shape

(13849, 17905)

In [25]:
users = df.userId.unique()
movies = df.movieId.unique()

userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [26]:
df['userId'] = df['userId'].apply(lambda x: userid2idx[x])
df['movieId'] = df['movieId'].apply(lambda x: movieid2idx[x])
split = np.random.rand(len(df)) < 0.8
train = df[split]
valid = df[~split]
print(train.shape , valid.shape)

(1607000, 4) (401784, 4)


In [27]:
n_movies=len(df['movieId'].unique())
n_users=len(df['userId'].unique())
n_latent_factors=64 

In [28]:
user_input=Input(shape=(1,),name='user_input',dtype='int64')

In [29]:
user_embedding=Embedding(n_users,n_latent_factors,name='user_embedding')(user_input)

In [30]:
user_vec =Flatten(name='FlattenUsers')(user_embedding)

In [31]:
movie_input=Input(shape=(1,),name='movie_input',dtype='int64')
movie_embedding=Embedding(n_movies,n_latent_factors,name='movie_embedding')(movie_input)
movie_vec=Flatten(name='FlattenMovies')(movie_embedding)

In [32]:
sim=dot([user_vec,movie_vec],name='Simalarity-Dot-Product',axes=1)
model =keras.models.Model([user_input, movie_input],sim)

In [33]:
model.compile(optimizer=Adam(lr=1e-4),loss='mse')

In [34]:
train.shape

(1607000, 4)

In [35]:
batch_size=512
epochs=5

In [36]:
History = model.fit([train.userId,train.movieId],train.rating, batch_size=batch_size,epochs =epochs,verbose = 1)

Epoch 1/5
3139/3139 [==============================] - 84s 26ms/step - loss: 13.5874
Epoch 2/5
3139/3139 [==============================] - 79s 25ms/step - loss: 12.8096
Epoch 3/5
3139/3139 [==============================] - 78s 25ms/step - loss: 7.2346
Epoch 4/5
3139/3139 [==============================] - 81s 26ms/step - loss: 3.3759
Epoch 5/5
3139/3139 [==============================] - 79s 25ms/step - loss: 2.1246


In [40]:
valid.shape

(401784, 4)

In [41]:
History_pict = model.predict([valid.userId,valid.movieId])

In [43]:
from math import sqrt


In [45]:
from sklearn.metrics import mean_squared_error


In [46]:
RMSE = sqrt(mean_squared_error(History_pict, valid.rating))

In [47]:
RMSE

1.3007007978632634